In [1]:
#import wrds
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
from scipy import stats
import pickle
from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import DateOffset
T=30

In [4]:
spx_front = pd.read_csv('data\\spx_front.csv').iloc[:,2:]
spx_next = pd.read_csv('data\\spx_next.csv').iloc[:,2:]
spx_spot = pd.read_csv('data\\spx_stock.csv',index_col=0,header=None)

spx_front = spx_front[~spx_front.duplicated()]
spx_next = spx_next[~spx_next.duplicated()]

spx_front.date = pd.to_datetime(spx_front.date)
spx_front.exdate = pd.to_datetime(spx_front.exdate)

spx_next.date = pd.to_datetime(spx_next.date)
spx_next.exdate = pd.to_datetime(spx_next.exdate)

spx_spot.index = pd.to_datetime(spx_spot.index)
spx_spot.columns = ["close"]

spx_dates = pd.to_datetime(sorted(spx_front.date.unique()))
spx_dates = spx_dates[spx_dates>=pd.to_datetime('1996-07-01')]

In [2]:
with open('../data/frontMonth.pickle', 'rb') as f:
    spx_front = pickle.load(f)
with open('../data/nextMonth.pickle','rb') as f:
    spx_next = pickle.load(f)
with open('../data/secp.pickle','rb') as f:
    spx_spot = pickle.load(f)   

In [58]:
spx_next.head()

,date,exdate,strike_price,impl_volatility,open_interest,cp_flag,best_bid,best_offer,forward_price
14,1996-01-04,1996-02-17,640.0,0.106798,5261.0,C,2.250,2.500,619.989341
32,1996-01-04,1996-02-17,620.0,0.119911,5621.0,P,9.750,10.250,619.989341
35,1996-01-04,1996-02-17,570.0,NaN,90.0,C,48.250,49.250,619.989341
37,1996-01-04,1996-02-17,630.0,0.123836,7391.0,P,15.750,16.500,619.989341
39,1996-01-04,1996-02-17,580.0,NaN,200.0,C,38.625,39.625,619.989341


In [3]:
def get_otm(optiondf,secdf):
    
    
    optiondf['option_price']=0.5*(optiondf['best_bid']+optiondf['best_offer'])
    optiondf.drop(columns=['best_bid', 'best_offer'],inplace=True)
    #optiondf['option_price']=optiondf['mid_price']
    optiondf['tao']=(optiondf['exdate']-optiondf['date']).apply(lambda x:x.days)
    
    
    

    #1.join spot and option price
    df1=optiondf.join(secdf,on='date')
    #optiondf has no index and secdf has index of date
    
    #2.join Call and put on the same day of same strike
    df1=df1.set_index(['date','strike_price'])

    calldf=df1[df1['cp_flag']=='C']
    putdf=df1[df1['cp_flag']=='P']
    df2=calldf.join(putdf['option_price'],how='inner',lsuffix='_call',rsuffix='_put')

    #3.select K which has the minimum absolute difference between call and put
    #and calculate F 
    df2['absdiff']=abs(df2['option_price_call']-df2['option_price_put'])
    df2['diff']=df2['option_price_call']-df2['option_price_put']
    df3=df2.loc[df2.groupby(['date'])['absdiff'].idxmin()]

    df3=df3.reset_index()
    df3['F']=df3['strike_price']+df3['forward_price']/df3['close']*df3['diff']

    #4.calculate K0
    df2=df2.reset_index()
    df3=df3.set_index(['date'])
    df4=df2.join(df3['F'],on='date',how='left')
    df5=df4[df4['F']-df4['strike_price']>0]
    df5['Fkdiff']=df5['F']-df5['strike_price']
    df6=df5.loc[df5.groupby(['date'])['Fkdiff'].idxmin()]
    df6['K0']=df6['strike_price']


    df6=df6.set_index(['date'])

    calldf=calldf.reset_index()
    calldf=calldf.set_index('date')
    calldff=calldf.join(df6['K0'],on='date',how='left')
    callotm=calldff[calldff['strike_price']>calldff['K0']]
    callitm=calldff[calldff['strike_price']<calldff['K0']]
    

    putdf=putdf.reset_index()
    putdf=putdf.set_index('date')
    putdff=putdf.join(df6['K0'],on='date',how='left')
    putotm=putdff[putdff['strike_price']<putdff['K0']]
    putitm=putdff[putdff['strike_price']>putdff['K0']]


    #df6:atm
    df6['option_price']=0.5*(df6['option_price_call']+df6['option_price_put'])
    df6=df6[callotm.columns]

    
    return callotm,putotm,callitm,putitm,df6

In [4]:
callotmf,putotmf,callitmf,putitmf,atmf = get_otm(spx_front,spx_spot)
callotmn,putotmn,callitmn,putitmn,atmn = get_otm(spx_next,spx_spot)

C:\Users\jiaxu\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
atmf.head()

,strike_price,exdate,impl_volatility,open_interest,cp_flag,forward_price,option_price,tao,close,K0
date,,,,,,,,,,
1996-01-04,615.0,1996-01-20,0.121763,4270.0,C,618.473789,6.56250,16,617.70,615.0
1996-01-05,615.0,1996-01-20,0.101374,4727.0,C,617.420695,5.40625,15,616.71,615.0
1996-01-08,615.0,1996-01-20,0.118369,4705.0,C,618.983149,5.12500,12,618.46,615.0
1996-01-09,600.0,1996-01-20,NaN,1087.0,C,609.942425,6.81250,11,609.45,600.0
1996-01-10,595.0,1996-01-20,0.158719,2521.0,C,598.898303,6.09375,10,598.48,595.0


In [6]:
allf=pd.concat([callotmf,putotmf,atmf],axis=0)#including otm call, otm put and atm options
alln=pd.concat([callotmn,putotmn,atmn],axis=0)

In [8]:
with open('../data/frontMonthalldf.pickle', 'wb') as f:
    pickle.dump(allf,f)
with open('../data/nextMonthalldf.pickle','wb') as f:
    pickle.dump(alln,f)

In [67]:
itmf=pd.concat([callitmf,putitmf],axis=0)##index:date
itmn=pd.concat([callitmn,putitmn],axis=0)

In [106]:
def createFeatures(space,num,atmdf,alldf):
    
    atmdf=atmdf[['exdate','strike_price','option_price']]
    df=pd.DataFrame()
    for d in range(len(atmdf)):
        d1=pd.DataFrame([atmdf.iloc[d,:]]*(2*num+1))
        d1['strike_price']=np.linspace(atmdf['strike_price'].iloc[d]-num*space,atmdf['strike_price'].iloc[d]+num*space,2*num+1)
        df=pd.concat([df,d1],axis=0)
    
    df=df.reset_index()

    alldf=alldf.reset_index()
    alldf=alldf.set_index(['date','strike_price'])
    
    featuredf=df.join(alldf['option_price'],on=['index','strike_price'],lsuffix='_temp')

    featuredf.drop(columns='option_price_temp',inplace=True)
    featuredf=featuredf.rename(columns={"index": "date"})

    return featuredf

In [109]:
featuredf_5f=createFeatures(5,5,atmf,allf)
featuredf_5n=createFeatures(5,5,atmn,alln)

In [111]:
featuredf_5f.head()

,date,exdate,strike_price,option_price
0,1996-01-04,1996-01-20,590.0,1.1250
1,1996-01-04,1996-01-20,595.0,1.5000
2,1996-01-04,1996-01-20,600.0,1.7500
3,1996-01-04,1996-01-20,605.0,2.6250
4,1996-01-04,1996-01-20,610.0,3.8125


In [73]:
featuredf_10f=createFeatures(5,10,atmf,allf)
featuredf_10n=createFeatures(5,10,atmn,alln)

In [99]:
def missing_describe(df):
    missingdf=df[df['option_price'].isnull()]
    missingdfbydate=missingdf.groupby('index').count()['strike_price']
    print("the maximum missing track at one day: %d"%(missingdfbydate.max()))
    print('the number of missing option price are %d in total'%(len(missingdf)))

In [100]:
missing_describe(featuredf_5)

the maximum missing track at one day: 10
the number of missing option price are 12768 in total


In [101]:
missing_describe(featuredf_10)

the maximum missing track at one day: 20
the number of missing option price are 27016 in total


# interpolate/expolate

In [194]:
type(featuredf_5f['date'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [195]:
featuredf_5f[featuredf_5f['date']==pd.datetime(1996,7,19)]

C:\Users\jiaxu\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  """Entry point for launching an IPython kernel.


,date,exdate,strike_price,option_price
1507,1996-07-19,1996-08-17,615.0,NaN
1508,1996-07-19,1996-08-17,620.0,4.6875
1509,1996-07-19,1996-08-17,625.0,5.9375
1510,1996-07-19,1996-08-17,630.0,7.3125
1511,1996-07-19,1996-08-17,635.0,8.5000
1512,1996-07-19,1996-08-17,640.0,10.7500
1513,1996-07-19,1996-08-17,645.0,8.4375
1514,1996-07-19,1996-08-17,650.0,6.2500
1515,1996-07-19,1996-08-17,655.0,4.3125
1516,1996-07-19,1996-08-17,660.0,3.0000


In [200]:
import numpy as np
from scipy import interpolate

def piecelinear(df,i,flag):
    print(i)
    if i==0:#missing first one
        print('first ex')
        
        while (i<len(df)-1) and np.isnan(df['option_price'].iloc[i]):
            i=i+1
        x=[df['strike_price'].iloc[i],df['strike_price'].iloc[i+1]]
        y=[df['option_price'].iloc[i],df['strike_price'].iloc[i+1]]
        f = interpolate.interp1d(x, y, fill_value='extrapolate')
        print(i)
        for s in range(i):
            df['option_price'].iloc[s]=f(df['strike_price'].iloc[s])
    
    else:
        
        start=i-1
        
        #find next non-nan
        while (i<len(df)-1) and np.isnan(df['option_price'].iloc[i]):
            i=i+1
        if i==(len(df)-1):
            print('last ex')
            #extropolate
            x=[df['strike_price'].iloc[start-1],df['strike_price'].iloc[start]]
            y=[df['option_price'].iloc[start-1],df['strike_price'].iloc[start]]
            f = interpolate.interp1d(x, y, fill_value='extrapolate')
            for s in range(start,(len(df)-1)):
                df['option_price'].iloc[s+1]=f(df['strike_price'].iloc[s+1])
            flag=True
        else:
            print('in')
            #interpolate
            x=[df['strike_price'].iloc[start],df['strike_price'].iloc[i]]
            y=[df['option_price'].iloc[start],df['strike_price'].iloc[i]]
        
            f = interpolate.interp1d(x, y, fill_value='extrapolate')
            df['option_price'].iloc[start+1]=f(df['strike_price'].iloc[start+1])
        
    
    return df,flag



In [201]:
def ixpolate(df):
    dfnew=pd.DataFrame()
    dfcopy=df.set_index('date')
    for s in dfcopy.index.unique():
        print(s)
        #for each day
        tempdf=dfcopy[dfcopy.index==s]
        flag=False
        for l in range(len(tempdf)):
            if np.isnan(tempdf['option_price'].iloc[l]):
                tempdf,flag=piecelinear(tempdf,l,flag)
            if flag:
                break
        dfnew=pd.concat([dfnew,tempdf],axis=0)
        
    return dfnew
                    
                

In [202]:
featuredf_5f_last=ixpolate(featuredf_5f)

1996-01-04 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-05 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-09 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-10 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-11 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-12 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-15 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-16 00:00:00
0
1
2
3
4
5
6
7
8
9
10
10
last ex
1996-01-17 00:00:00
0
1
2
3
4
5
6
7
8
9
10
10
last ex
1996-01-18 00:00:00
0
1
2
3
4
5
6
7
8
9
10
10
last ex
1996-01-19 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-22 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-23 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-24 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-25 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-26 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-29 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-30 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-01-31 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-02-01 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-02-02 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-02-05 00:00:00
0

C:\Users\jiaxu\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2
3
4
5
6
7
8
9
10
1996-04-15 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-16 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-17 00:00:00
0
1
2
3
4
5
6
7
8
9
10
10
last ex
1996-04-18 00:00:00
0
1
2
3
4
5
6
7
8
9
9
last ex
1996-04-19 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-22 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-23 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-24 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-25 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-26 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-29 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-04-30 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-01 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-02 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-03 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-06 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-07 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-09 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-10 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-13 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-14 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-05-15 00:00

C:\Users\jiaxu\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()



8
9
10
1996-07-25 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-07-26 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-07-29 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-07-30 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-07-31 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-01 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-02 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-05 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-06 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-07 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-09 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-12 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-13 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-14 00:00:00
0
1
2
3
4
5
6
7
8
9
10
10
last ex
1996-08-15 00:00:00
0
1
2
3
4
5
6
7
8
9
9
last ex
1996-08-16 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-19 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-20 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-21 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-22 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-23 00:00:00
0
1
2
3
4
5
6
7
8
9
10
1996-08-26 00:00:00
0
1
2
3


...

2016-07-13 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-14 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-15 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-18 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-19 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-20 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-21 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-22 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-25 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-26 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-27 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-28 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-07-29 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-01 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-02 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-03 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-04 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-05 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-09 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-10 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-11 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-12 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2016-08-15 

0
1
2
3
4
5
6
7
8
9
10
2017-06-01 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-02 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-05 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-06 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-07 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-09 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-12 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-13 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-14 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-06-15 00:00:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
1

0
1
2
3
4
5
6
7
8
9
10
2017-12-21 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-12-22 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-12-26 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-12-27 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-12-28 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2017-12-29 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-02 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-03 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-04 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-05 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-09 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-10 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-11 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-12 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-16 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-17 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-01-18 00:00:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78

0
1
2
3
4
5
6
7
8
9
10
2018-07-09 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-10 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-11 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-12 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-13 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-16 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-17 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-18 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-19 00:00:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2018-07-20 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-23 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-24 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-25 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-26 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-27 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-30 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-07-31 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-08-01 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-08-02 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-08-03 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-08-06 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2018-08-07 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2

0
1
2
3
4
5
6
7
8
9
10
2019-02-06 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-07 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-11 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-12 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-13 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-14 00:00:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2019-02-15 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-19 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-20 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-21 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-22 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-25 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-26 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-27 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-02-28 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-03-01 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-03-04 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-03-05 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-03-06 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-03-07 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2019-03-08 00:00:00
0
1
2
3
4
5
6
7
8
9
10
2

In [204]:
featuredf_5f_last[featuredf_5f_last['option_price'].isnull()]

,exdate,strike_price,option_price
date,,,


# Call Put Front Next

In [75]:
def get_idate_opt(callotmf,idate):
    ioptcf = callotmf[callotmf.index == idate]

    ioptcf.sort_values(by="strike_price",inplace = True)

    ioptcf.set_index("strike_price",inplace=True)
    return ioptcf

In [104]:
def get_features(callotmf,callotmn,putotmf,putotmn,n):
    feature_list = []

    for i in range(len(spx_dates)):
        idate = spx_dates[i]
        print(idate)
        ioptcf = get_idate_opt(callotmf,idate)
        ioptcn = get_idate_opt(callotmn,idate)

        ioptpf = get_idate_opt(putotmf,idate)
        ioptpn = get_idate_opt(putotmn,idate)

        concat_call = ioptcf.join(ioptcn,how='inner',lsuffix='_f',rsuffix='_n')

        concat_call['weighted_price'] = ((concat_call["tao_n"]-T)*concat_call["mid_price_f"]+(T-concat_call["tao_f"])*concat_call["mid_price_n"])/(concat_call["tao_n"]-concat_call["tao_f"])

        concat_put = ioptpf.join(ioptpn,how='inner',lsuffix='_f',rsuffix='_n')

        concat_put['weighted_price'] = (concat_put["tao_n"]-T)*concat_put["mid_price_f"]+((T-concat_put["tao_f"])*concat_put["mid_price_n"])/(concat_put["tao_n"] - concat_put["tao_f"])

        concat_all = pd.concat([concat_put,concat_call],axis=0)
        concat_all["abs_diff"] = abs(concat_all.index.values - concat_all.K0_f.values)
        try:
            min_iloc = concat_all.index.get_loc(concat_all.abs_diff.idxmin())
            features = concat_all.iloc[min_iloc-n:min_iloc+n+1,:]["weighted_price"].values
        except:
            features = [np.nan]*(2*n+1)
            
        feature_list.append(features)
        feature_df = pd.DataFrame(feature_list)

    return feature_df

In [105]:
feature_df = get_features(callotmf,callotmn,putotmf,putotmn,5)

2015-07-10 00:00:00
2015-07-13 00:00:00
2015-07-14 00:00:00
2015-07-15 00:00:00
2015-07-16 00:00:00
2015-07-17 00:00:00
2015-07-20 00:00:00
2015-07-21 00:00:00
2015-07-22 00:00:00
2015-07-23 00:00:00
2015-07-24 00:00:00
2015-07-27 00:00:00
2015-07-28 00:00:00
2015-07-29 00:00:00
2015-07-30 00:00:00
2015-07-31 00:00:00
2015-08-03 00:00:00
2015-08-04 00:00:00
2015-08-05 00:00:00
2015-08-06 00:00:00
2015-08-07 00:00:00
2015-08-10 00:00:00
2015-08-11 00:00:00
2015-08-12 00:00:00
2015-08-13 00:00:00
2015-08-14 00:00:00
2015-08-17 00:00:00
2015-08-18 00:00:00
2015-08-19 00:00:00
2015-08-20 00:00:00
2015-08-21 00:00:00
2015-08-24 00:00:00
2015-08-25 00:00:00
2015-08-26 00:00:00
2015-08-27 00:00:00
2015-08-28 00:00:00
2015-08-31 00:00:00
2015-09-01 00:00:00
2015-09-02 00:00:00
2015-09-03 00:00:00
2015-09-04 00:00:00
2015-09-08 00:00:00
2015-09-09 00:00:00
2015-09-10 00:00:00
2015-09-11 00:00:00
2015-09-14 00:00:00
2015-09-15 00:00:00
2015-09-16 00:00:00
2015-09-17 00:00:00
2015-09-18 00:00:00


In [137]:
feature_df.index  = spx_dates

In [139]:
feature_df.to_csv("features_11.csv")

## Vol target

In [14]:
spx_return = spx_spot.pct_change()+1

spx_log_return = np.log(spx_return)
spx_var = spx_log_return.copy()
spx_var.iloc[:,0] =  np.nan

for i in range(len(spx_log_return)-30):
    var = np.var(spx_log_return.iloc[i:i+30,0])
    date = spx_log_return.index[i]
    spx_var.iloc[i,0] = var

spx_spot["log_return"] = spx_log_return.values
spx_spot["variance_30"] = spx_var.values
spx_spot["vol_30"] = np.sqrt(spx_var.values)
spx_spot["ann_vol_30"] = np.sqrt(spx_var.values) * np.sqrt(252) *100
spx_spot.to_csv("data/spx_vol.csv")

## vix

In [15]:
vix = pd.read_csv("data/vix.csv",index_col=0)
vix.index = pd.to_datetime(vix.index)
spx_spot= spx_spot.join(vix["Close"],how="inner",rsuffix="_vix")
spx_spot["vix"] = spx_spot["Close"]
spx_target = spx_spot.loc[:,["ann_vol_30","vix"]]

spx_target.to_csv("data/spx_target.csv")